---

# <center> GFootball Stable-Baselines3 </center>

---
<center><img src="https://raw.githubusercontent.com/DLR-RM/stable-baselines3/master/docs/_static/img/logo.png" width="308" height="268" alt="Stable-Baselines3"></center>
<center><small>Image from Stable-Baselines3 repository</small></center>

---
This notebook uses the [Stable-Baselines3](https://github.com/DLR-RM/stable-baselines3) library to train a [PPO](https://openai.com/blog/openai-baselines-ppo/) reinforcement learning agent on [GFootball Academy](https://github.com/google-research/football/tree/master/gfootball/scenarios) scenarios, applying the architecture from the paper "[Google Research Football: A Novel Reinforcement Learning Environment](https://arxiv.org/abs/1907.11180)".

In [1]:
%%bash
# dependencies
apt-get -y update > /dev/null
apt-get -y install libsdl2-gfx-dev libsdl2-ttf-dev > /dev/null

# cloudpickle, pytorch, gym
pip3 install "cloudpickle==1.3.0"
pip3 install "torch==1.5.1"
pip3 install "gym==0.17.2"

# gfootball
GRF_VER=v2.8
GRF_PATH=football/third_party/gfootball_engine/lib
GRF_URL=https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_${GRF_VER}.so
git clone -b ${GRF_VER} https://github.com/google-research/football.git
mkdir -p ${GRF_PATH}
wget -q ${GRF_URL} -O ${GRF_PATH}/prebuilt_gameplayfootball.so
cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install . && cd ..

# kaggle-environments
git clone https://github.com/Kaggle/kaggle-environments.git
cd kaggle-environments && pip3 install . && cd ..

# stable-baselines3
git clone https://github.com/DLR-RM/stable-baselines3.git
cd stable-baselines3 && pip3 install . && cd ..

# housekeeping
# rm -rf football kaggle-environments stable-baselines3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for gym: filename=gym-0.17.2-py3-none-any.whl size=1650890 sha256=3fcb496082d093ee5b4901f4da4bca032167c4eddd9107c471a1bbbe779ef26c
  Stored in directory: /root/.cache/pip/wheels/18/e1/58/89a2aa24e6c2cc800204fc02010612afdf200926c4d6bfe315
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /con

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.5.1 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.5.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.5.1 which is incompatible.
Cloning into 'football'...
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
Cloning into 'kaggle-environments'...
  DEPRECATION: A future pip version will change local packages to be built in-plac

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/My Drive/Colab')

Mounted at /content/drive


In [3]:
import os
import base64
import pickle
import zlib
import gym
import numpy as np
import pandas as pd
import torch as th
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from torch import nn, tensor
from collections import deque
from gym.spaces import Box, Discrete
from kaggle_environments import make
from kaggle_environments.envs.football.helpers import *
from gfootball.env import create_environment, observation_preprocessing
from stable_baselines3 import PPO
from stable_baselines3.ppo import CnnPolicy
from stable_baselines3.common import results_plotter
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.vec_env.subproc_vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

from IPython.display import HTML
# from visualizer import visualize

from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
# prev_obs is the obs before the action is taken, obs is the next obs as a result of the action.
def reward_modifier(rew, action, prev_obs, obs):
  
  ball_x, ball_y, ball_z = obs['ball']
  MIDDLE_X, PENALTY_X, END_X = 0.2, 0.64, 1.0
  PENALTY_Y, END_Y = 0.27, 0.42

  # Ball position
  ball_position_r = 0.0
  if   (-END_X <= ball_x    and ball_x < -PENALTY_X)and (-PENALTY_Y < ball_y and ball_y < PENALTY_Y):
      ball_position_r = -2.0
  elif (-END_X <= ball_x    and ball_x < -MIDDLE_X) and (-END_Y < ball_y     and ball_y < END_Y):
      ball_position_r = -1.0
  elif (-MIDDLE_X <= ball_x and ball_x <= MIDDLE_X) and (-END_Y < ball_y     and ball_y < END_Y):
      ball_position_r = 0.0
  elif (PENALTY_X < ball_x  and ball_x <=END_X)     and (-PENALTY_Y < ball_y and ball_y < PENALTY_Y):
      ball_position_r = 2.0
  elif (MIDDLE_X < ball_x   and ball_x <=END_X)     and (-END_Y < ball_y     and ball_y < END_Y):
      ball_position_r = 1.0
  else:
      ball_position_r = 0.0

  # Yellow card 
  left_yellow = np.sum(obs["left_team_yellow_card"]) -  np.sum(prev_obs["left_team_yellow_card"])
  right_yellow = np.sum(obs["right_team_yellow_card"]) -  np.sum(prev_obs["right_team_yellow_card"])
  yellow_r = right_yellow - left_yellow
  
  # Score 
  win_reward = 0.0
  if obs['steps_left'] == 0:
      [my_score, opponent_score] = obs['score']
      if my_score > opponent_score:
          win_reward = 1.0

  # Possession of ball
  # loses ball to opponent
  possession_r = 0
  if (prev_obs['ball_owned_team'] == 0) and (obs['ball_owned_team'] == 1):
    possession_r = -1
  # steals ball
  elif (prev_obs['ball_owned_team'] != 0) and (obs['ball_owned_team'] == 0):
    possession_r = 1

  # Corner kicks and Penalty kicks

  return 5.0*rew + 4.0*win_reward + 0.003*ball_position_r + yellow_r + 0.003*possession_r


---
# Football Gym
> [Stable-Baselines3: Custom Environments](https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html)<br/>
> [SEED RL Agent](https://www.kaggle.com/piotrstanczyk/gfootball-train-seed-rl-agent): stacked observations

In [5]:
class FootballGym(gym.Env):
    spec = None
    metadata = None
    
    def __init__(self, config=None):
        super(FootballGym, self).__init__()
        env_name = "academy_empty_goal_close"
        rewards = "scoring,checkpoints"
        self.reward_mod = True
        if config is not None:
            env_name = config.get("env_name", env_name)
            rewards = config.get("rewards", rewards)
            self.reward_mod = config.get("reward_mod", self.reward_mod)
        self.env = create_environment(
            env_name=env_name,
            stacked=False,
            representation="raw",
            rewards = rewards,
            write_goal_dumps=False,
            write_full_episode_dumps=False,
            render=False,
            write_video=False,
            dump_frequency=1,
            logdir=".",
            extra_players=None,
            number_of_left_players_agent_controls=1,
            number_of_right_players_agent_controls=0)  
        self.action_space = Discrete(19)
        self.observation_space = Box(low=0, high=255, shape=(72, 96, 16), dtype=np.uint8)
        # self.reward_range = (-1, 1)
        self.obs_stack = deque([], maxlen=4)
        self.prev_obs = self.observation_space.sample()[0]

  # elif representation == 'simple115':
  #   env = wrappers.Simple115StateWrapper(env)
  # elif representation == 'simple115v2':
  #   env = wrappers.Simple115StateWrapper(env, True)
  # elif representation == 'extracted':
  #   env = wrappers.SMMWrapper(env, channel_dimensions)
  # elif representation == 'raw':

    def transform_obs(self, raw_obs):
        obs = raw_obs[0]
        obs = observation_preprocessing.generate_smm([obs])
        if not self.obs_stack:
            self.obs_stack.extend([obs] * 4)
        else:
            self.obs_stack.append(obs)

        obs = np.concatenate(list(self.obs_stack), axis=-1)
        obs = np.squeeze(obs)
        return obs

    def reset(self):
        self.obs_stack.clear()
        obs = self.env.reset()
        self.prev_obs = obs[0]
        obs = self.transform_obs(obs)
        return obs
    
    def step(self, action):
        obs, reward, done, info = self.env.step([action])
        final_reward = reward
        if self.reward_mod == True:
          final_reward = reward_modifier(reward, action, self.prev_obs, obs[0])
        self.prev_obs = obs[0]
        obs = self.transform_obs(obs)
        return obs, float(final_reward), done, info
    
check_env(env=FootballGym(), warn=True)

---
# Football CNN
> [Stable-Baselines3: Custom Policy Network](https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html)<br/>
> [Google Research Football: A Novel Reinforcement Learning Environment](https://arxiv.org/abs/1907.11180)

In [6]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=True)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.conv2 = conv3x3(out_channels, out_channels, stride)
        
    def forward(self, x):
        residual = x
        out = self.relu(x)
        out = self.conv1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out += residual
        return out
    
class FootballCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super().__init__(observation_space, features_dim)
        in_channels = observation_space.shape[0]  # channels x height x width
        self.cnn = nn.Sequential(
            conv3x3(in_channels=in_channels, out_channels=32),
            nn.MaxPool2d(kernel_size=3, stride=2, dilation=1, ceil_mode=False),
            ResidualBlock(in_channels=32, out_channels=32),
            ResidualBlock(in_channels=32, out_channels=32),
            nn.ReLU(),
            nn.Flatten(),
        )
        self.linear = nn.Sequential(
          nn.Linear(in_features=52640, out_features=features_dim, bias=True),
          nn.ReLU(),
        )

    def forward(self, obs):
        return self.linear(self.cnn(obs))

---
# PPO Model
> [Stable-Baselines3: PPO](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html)<br/>
> [Stable-Baselines3: Vectorized Environments](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html)<br/>
> [Stable-Baselines3: Custom Policy Network](https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html)<br/>
> [GFootball: A Novel Reinforcement Learning Environment](https://arxiv.org/abs/1907.11180)<br/>
> [GFootball: Academy Scenarios](https://github.com/google-research/football/tree/master/gfootball/scenarios)<br/>

In [7]:
scenarios = {0: "academy_empty_goal_close",
             1: "academy_empty_goal",
             2: "academy_run_to_score",
             3: "academy_run_to_score_with_keeper",
             4: "academy_pass_and_shoot_with_keeper",
             5: "academy_run_pass_and_shoot_with_keeper",
             6: "academy_3_vs_1_with_keeper",
             7: "academy_corner",
             8: "academy_counterattack_easy",
             9: "academy_counterattack_hard",
             10: "academy_single_goal_versus_lazy",
             11: "11_vs_11_kaggle"}
scenario_name = scenarios[7]

In [8]:
filepath = "/content/drive/My Drive/RL/Final_project/trained_agent"
experiment = "cha_FINAL3"
log_dir = f"{filepath}/{experiment}"

def make_env(config=None, rank=0):
    # def _init():
    env = FootballGym(config)
    # env = wrap_env(env,0,0)
    env = Monitor(env, log_dir#, allow_early_resets=True#
                  )
    return env
    # return _init

In [9]:
# class FootballOpponentEnv(gym.Wrapper):
#   def __init__(self, env, opponent):
#     gym.Wrapper.__init__(self, env)
#     self.policy = self
#     if type(opponent) is not list:
#       opponent = [opponent]
#     self.opponent_list = opponent
#     self.opponent_list = self.opponent_list + [BaselinePolicy()]
#     self.curr_index=1
#     self.opponent = self.opponent_list[self.curr_index]


#   def predict(self, obs): # the policy
#     action = self.opponent.predict(obs)
#     if self.curr_index == 0:
#       print(self.opponent)
#       print(self.otherAction)
#       return action[0]
#     return action

#   def reset(self):
#     global checkpoint
#     global curr_reward
#     if checkpoint:
#       checkpoint = False
#       if(threshmap[self.curr_index] < curr_reward):
#         if self.curr_index ==1:
#           newindex = 0
#         else:
#           newindex = self.curr_index+1
#         print("!!!opponent changed from"+str(self.curr_index)+"to"+str(newindex))
#         self.curr_index = newindex
#         self.opponent = self.opponent_list[self.curr_index]

#     return super(SlimeVolleyOpponentEnv, self).reset()

In [10]:
training_scenarios = {0: "11_vs_11_easy_stochastic",
             1: "11_vs_11_stochastic",
             2: "11_vs_11_hard_stochastic",
             3: "11_vs_11_kaggle"}
training_scenario_name = training_scenarios[1]

In [11]:
class FootballActionEnv(gym.ActionWrapper):
  def __init__(self, env):
    gym.ActionWrapper.__init__(self, env)


  def action(self, act):
    obs = self.env.env.env.unwrapped.observation()
    obs = obs[0]
    player_num = obs['active']

    player_pos_x, player_pos_y = obs['left_team'][player_num]
    ball_x, ball_y, ball_z = obs['ball']
    ball_x_relative = ball_x - player_pos_x
    ball_y_relative = ball_y - player_pos_y

    ball_distance = np.linalg.norm([ball_x_relative, ball_y_relative])

    act = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    NO_OP, LEFT, TOP_LEFT, TOP, TOP_RIGHT, RIGHT, BOTTOM_RIGHT, BOTTOM, BOTTOM_LEFT, LONG_PASS, HIGH_PASS, SHORT_PASS, SHOT, SPRINT, RELEASE_DIRECTION, \
                          RELEASE_SPRINT, SLIDE, DRIBBLE, RELEASE_DRIBBLE = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18

    if obs['ball_owned_team'] == 1: # opponents owning ball
      act[LONG_PASS], act[HIGH_PASS], act[SHORT_PASS], act[SHOT], act[DRIBBLE] = 0, 0, 0, 0, 0
    elif obs['ball_owned_team'] == -1 and ball_distance > 0.03 and obs['game_mode'] == 0: # Ground ball  and far from me
      act[LONG_PASS], act[HIGH_PASS], act[SHORT_PASS], act[SHOT], act[DRIBBLE] = 0, 0, 0, 0, 0
    else: # my team owning ball
      act[SLIDE] = 0

    # Dealing with sticky actions
    sticky_actions = obs['sticky_actions']
    if sticky_actions[8] == 0:  # sprinting
      act[RELEASE_SPRINT] = 0
            
    if sticky_actions[9] == 1:  # dribbling
      act[SLIDE] = 0
    else:
      act[RELEASE_DRIBBLE] = 0

    if np.sum(sticky_actions[:8]) == 0:
      act[RELEASE_DIRECTION] = 0

    ball_x, ball_y, _ = obs['ball']
    if ball_x < 0.64 or ball_y < -0.27 or 0.27 < ball_y:
      act[SHOT] = 0
    elif (0.64 <= ball_x and ball_x<=1.0) and (-0.27<=ball_y and ball_y<=0.27):
      act[HIGH_PASS], act[LONG_PASS] = 0, 0

    if obs['game_mode'] == 2 and ball_x < -0.7:  # Our GoalKick 
      act = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
      act[LONG_PASS], act[HIGH_PASS], act[SHORT_PASS] = 1, 1, 1
      return act.index(1)

    elif obs['game_mode'] == 4 and ball_x > 0.9:  # Our CornerKick
      act = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
      act[LONG_PASS], act[HIGH_PASS], act[SHORT_PASS] = 1, 1, 1
      return act.index(1)

    elif obs['game_mode'] == 6 and ball_x > 0.6:  # Our PenaltyKick
      act = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
      act[SHOT] = 1
      return act.index(1)

    return act.index(1)


n_envs = 4
config={"env_name":training_scenario_name, "rewards":"scoring,checkpoints", "reward_mod": True}
# train_env = DummyVecEnv([make_env(config, rank=i) for i in range(n_envs)])
# train_env = SubprocVecEnv([make_env(config, rank=i) for i in range(n_envs)])
train_env = make_vec_env(FootballGym,n_envs=4,env_kwargs=dict(config=config),monitor_dir=log_dir, wrapper_class = FootballActionEnv)

n_steps = 512
policy_kwargs = dict(features_extractor_class=FootballCNN,
                     features_extractor_kwargs=dict(features_dim=256))
# model = PPO(CnnPolicy, train_env, 
#             policy_kwargs=policy_kwargs, 
#             learning_rate=0.000343, 
#             n_steps=n_steps, 
#             batch_size=8, 
#             n_epochs=2, 
#             gamma=0.993,
#             gae_lambda=0.95,
#             clip_range=0.08, 
#             ent_coef=0.003, 
#             vf_coef=0.5, 
#             max_grad_norm=0.64, 
#             verbose=2,
#             tensorboard_log=f"{filepath}/tensorboard")
model = PPO.load("/content/drive/My Drive/RL/Final_project/trained_agent/cha_FINAL2/best_model.zip", train_env)

Wrapping the env in a VecTransposeImage.


---
# Training
> [Stable-Baselines3: Examples](https://stable-baselines3.readthedocs.io/en/master/guide/examples.html)<br/>
> [Stable-Baselines3: Callbacks](https://stable-baselines3.readthedocs.io/en/master/guide/callbacks.html)

In [12]:

training_index=0
checkpoint = False
from tqdm.notebook import tqdm
class SaveonBestandProgressBar(BaseCallback):
    def __init__(self, check_freq: int, log_dir: str, verbose: int = 1):
        super(SaveonBestandProgressBar, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf
        self.pbar = None

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_training_start(self):
        factor = np.ceil(self.locals['total_timesteps'] / self.model.n_steps)
        n = 1
        try:
            n = len(self.training_env.envs)
        except AttributeError:
            try:
                n = len(self.training_env.remotes)
            except AttributeError:
                n = 1
        total = int(self.model.n_steps * factor / n)
        self.pbar = tqdm(total=total)

    def _on_rollout_start(self):
        self.pbar.refresh()

    def _on_step(self):
        self.pbar.update(1)
        if self.n_calls % self.check_freq == 0:
          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), 'timesteps')
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose > 0:
                print(f"Num timesteps: {self.num_timesteps}")
                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose > 0:
                    print(f"Saving new best model to {self.save_path}")
                  self.model.save(self.save_path)

        return True

    def _on_rollout_end(self):
        # if self.best_mean_reward > 0:
        #   global training_index
        #   global training_scenarios
        #   if training_index != len(training_scenarios)-1:
        #     training_index+=1
        #     scenario_name = training_scenarios[training_index]
        #     print(scenario_name)
        #     config={"env_name":scenario_name}
        #     train_env = make_vec_env(FootballGym,n_envs=4,env_kwargs=dict(config=config),monitor_dir=log_dir)
        #     self.model.set_env(train_env)
        #     self.best_mean_reward = -np.inf
        self.pbar.refresh()

    def _on_training_end(self):
        self.pbar.close()
        self.pbar = None

Callbacktinst = SaveonBestandProgressBar(check_freq=10000, log_dir=log_dir)

In [13]:
total_epochs = 1000
total_timesteps = n_steps * n_envs * total_epochs
# t_scenario_name = training_scenarios[0]
# t_config={"env_name":t_scenario_name}
# t_train_env = make_vec_env(FootballGym,n_envs=4,env_kwargs=dict(config=t_config),monitor_dir=log_dir)
# model.set_env(t_train_env)
print(vars(model.env.envs[0].env.env).keys())
model.learn(total_timesteps=total_timesteps, callback=Callbacktinst, log_interval=None)
model.save("ppo_gfootball")

dict_keys(['reward_mod', 'env', 'action_space', 'observation_space', 'reward_range', 'obs_stack', 'prev_obs'])
Logging to /content/drive/My Drive/RL/Final_project/trained_agent/tensorboard/PPO_9


  0%|          | 0/512000 [00:00<?, ?it/s]

Num timesteps: 40000
Best mean reward: -inf - Last mean reward per episode: -13.04
Saving new best model to /content/drive/My Drive/RL/Final_project/trained_agent/cha_FINAL2/best_model
Num timesteps: 80000
Best mean reward: -13.04 - Last mean reward per episode: -14.45
Num timesteps: 120000
Best mean reward: -13.04 - Last mean reward per episode: -14.11
Num timesteps: 160000
Best mean reward: -13.04 - Last mean reward per episode: -13.86
Num timesteps: 200000
Best mean reward: -13.04 - Last mean reward per episode: -14.03
Num timesteps: 240000
Best mean reward: -13.04 - Last mean reward per episode: -13.96
Num timesteps: 280000
Best mean reward: -13.04 - Last mean reward per episode: -13.60
Num timesteps: 320000
Best mean reward: -13.04 - Last mean reward per episode: -13.75
Num timesteps: 360000
Best mean reward: -13.04 - Last mean reward per episode: -13.85
Num timesteps: 400000
Best mean reward: -13.04 - Last mean reward per episode: -13.65
Num timesteps: 440000
Best mean reward: -1

KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "{filepath}/tensorboard"

In [ ]:
plt.style.use(['seaborn-whitegrid'])
results_plotter.plot_results([log_dir], total_timesteps, results_plotter.X_TIMESTEPS, "GFootball Timesteps")
results_plotter.plot_results([log_dir], total_timesteps, results_plotter.X_EPISODES, "GFootball Episodes")

In [ ]:
plt.style.use(['seaborn-whitegrid'])
log_files = [os.path.join(log_dir, f"{i}.monitor.csv") for i in range(n_envs)]

nrows = np.ceil(n_envs/2)
fig = plt.figure(figsize=(8, 2 * nrows))
for i, log_file in enumerate(log_files):
    if os.path.isfile(log_file):
        df = pd.read_csv(log_file, skiprows=1)
        plt.subplot(nrows, 2, i+1, label=log_file)
        df['r'].rolling(window=100).mean().plot(title=f"Rewards: Env {i}")
        plt.tight_layout()
plt.show()

In [ ]:
%%writefile submission.py
import base64
import pickle
import zlib
import numpy as np
import torch as th
from torch import nn, tensor
from collections import deque
from gfootball.env import observation_preprocessing

state_dict = _STATE_DICT_

state_dict = pickle.loads(zlib.decompress(base64.b64decode(state_dict)))

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=True)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.conv2 = conv3x3(out_channels, out_channels, stride)
        
    def forward(self, x):
        residual = x
        out = self.relu(x)
        out = self.conv1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out += residual
        return out
    
class PyTorchCnnPolicy(nn.Module):
    global state_dict
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            conv3x3(in_channels=16, out_channels=32),
            nn.MaxPool2d(kernel_size=3, stride=2, dilation=1, ceil_mode=False),
            ResidualBlock(in_channels=32, out_channels=32),
            ResidualBlock(in_channels=32, out_channels=32),
            nn.ReLU(),
            nn.Flatten(),
        )
        self.linear = nn.Sequential(
          nn.Linear(in_features=52640, out_features=256, bias=True),
          nn.ReLU(),
        )
        self.policy_net = nn.Sequential(
          nn.Linear(in_features=256, out_features=64, bias=True),
          nn.Tanh(),
          nn.Linear(in_features=64, out_features=64, bias=True),
          nn.Tanh(),
        )
        self.action_net = nn.Sequential(
          nn.Linear(in_features=64, out_features=19, bias=True),
          nn.ReLU(),
        )
        self.out_activ = nn.Softmax(dim=1)
        self.load_state_dict(state_dict)

    def forward(self, x):
        x = tensor(x).float() / 255.0  # normalize
        x = x.permute(0, 3, 1, 2).contiguous()  # 1 x channels x height x width
        x = self.cnn(x)
        x = self.linear(x)
        x = self.policy_net(x)
        x = self.action_net(x)
        x = self.out_activ(x)
        return int(x.argmax())
    
obs_stack = deque([], maxlen=4)
def transform_obs(raw_obs):
    global obs_stack
    obs = raw_obs['players_raw'][0]
    obs = observation_preprocessing.generate_smm([obs])
    if not obs_stack:
        obs_stack.extend([obs] * 4)
    else:
        obs_stack.append(obs)
    obs = np.concatenate(list(obs_stack), axis=-1)
    return obs

policy = PyTorchCnnPolicy()
policy = policy.float().to('cpu').eval()
def agent(raw_obs):
    obs = transform_obs(raw_obs)
    action = policy(obs)
    return [action]

In [ ]:
model = PPO.load("ppo_gfootball")
_state_dict = model.policy.to('cpu').state_dict()
state_dict = {
    "cnn.0.weight":_state_dict['features_extractor.cnn.0.weight'], 
    "cnn.0.bias":_state_dict['features_extractor.cnn.0.bias'], 
    "cnn.2.conv1.weight":_state_dict['features_extractor.cnn.2.conv1.weight'], 
    "cnn.2.conv1.bias":_state_dict['features_extractor.cnn.2.conv1.bias'],
    "cnn.2.conv2.weight":_state_dict['features_extractor.cnn.2.conv2.weight'], 
    "cnn.2.conv2.bias":_state_dict['features_extractor.cnn.2.conv2.bias'], 
    "cnn.3.conv1.weight":_state_dict['features_extractor.cnn.3.conv1.weight'], 
    "cnn.3.conv1.bias":_state_dict['features_extractor.cnn.3.conv1.bias'], 
    "cnn.3.conv2.weight":_state_dict['features_extractor.cnn.3.conv2.weight'], 
    "cnn.3.conv2.bias":_state_dict['features_extractor.cnn.3.conv2.bias'], 
    "linear.0.weight":_state_dict['features_extractor.linear.0.weight'], 
    "linear.0.bias":_state_dict['features_extractor.linear.0.bias'], 
    "policy_net.0.weight":_state_dict['mlp_extractor.policy_net.0.weight'],
    "policy_net.0.bias":_state_dict['mlp_extractor.policy_net.0.bias'],
    "policy_net.2.weight":_state_dict['mlp_extractor.policy_net.2.weight'],
    "policy_net.2.bias":_state_dict['mlp_extractor.policy_net.2.bias'],
    "action_net.0.weight":_state_dict['action_net.weight'],
    "action_net.0.bias":_state_dict['action_net.bias'],
}
state_dict = base64.b64encode(zlib.compress(pickle.dumps(state_dict)))
with open('submission.py', 'r') as file:
    src = file.read()
src = src.replace("_STATE_DICT_", f"{state_dict}")
with open('submission.py', 'w') as file:
    file.write(src)

---
# Testing

In [ ]:
from stable_baselines3.common.monitor import Monitor as TrainMonitor
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import io
import base64
from IPython.display import HTML, clear_output
from IPython import display as ipythondisplay
from gym.wrappers import Monitor as eval_Monitor

def show_video(episode, rnd):
  mp4list = glob.glob(f'video_{episode}_{rnd}/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    os.system(f"ffmpeg -i {mp4} -vcodec libx264 video_{episode}_{rnd}/compressed.mp4")
    video = io.open(f'video_{episode}_{rnd}/compressed.mp4', 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 200px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

def wrap_env(env, episode, rnd):
    env = eval_Monitor(env, f'./video_{episode}_{rnd}', force=True)
    return env

In [ ]:
# model = PPO.load("ppo_gfootball")
config={"env_name":training_scenario_name, "rewards":"scoring", "reward_mod": False}
test_env = FootballGym(config)
obs = test_env.reset()
done = False
total_reward=0
while not done:
    action, state = model.predict(obs, deterministic=True)
    obs, reward, done, info = test_env.step(action)
    total_reward+=reward

    # print(f"{Action(action).name.ljust(16,' ')}\t{round(reward,2)}\t{info}")
print(total_reward)

In [ ]:
kaggle_env = make("football", debug = False,
                  configuration={"scenario_name": training_scenario_name, 
                                 "running_in_notebook": True,
                                 "save_video": True})
output = kaggle_env.run(["submission.py", "run_right"])
print(output)
scores = output[-1][0]["observation"]["players_raw"][0]["score"]
print("Scores  {0} : {1}".format(*scores))
print("Rewards {0} : {1}".format(output[-1][0]["reward"], output[-1][1]["reward"]))

viz = visualize(output)


# # viz = visualize(output)
HTML(viz.to_html5_video())

# env_name = "11_vs_11_kaggle"
# rewards = "scoring,checkpoints"

# # p=wrap_env(create_environment(env_name),0,0)
# # test_env = DummyVecEnv([make_env()])
# model = PPO.load("ppo_gfootball")
# obs = test_env.reset()
# done = False
# while not done:
#     action, state = model.predict(obs, deterministic=True)
#     obs, reward, done, info = test_env.step(action)
#     # print(f"{Action(action).name.ljust(16,' ')}\t{round(reward,2)}\t{info}")


# # show_video(0, 0)
 




> Modified [Human Readable Visualization](https://www.kaggle.com/jaronmichal/human-readable-visualization)

In [ ]:
env = make("football", configuration={"save_video": True, "scenario_name": training_scenario_name, "running_in_notebook": True}, debug=True)
agent = "submission.py"
output = env.run([agent, "run_right"])[-1]
print('Left player: action = %s, reward = %s, status = %s, info = %s' % (output[0]["action"], output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: action = %s, reward = %s, status = %s, info = %s' % (output[1]["action"], output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)

---
# Checkpoints

0. [academy_empty_goal_close @ 800K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=45569809#Test-Agent) (Nature CNN)<br/>
1. [academy_empty_goal @ 800K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=45639135#Test-Agent) (Nature CNN)<br/>
2. [academy_run_to_score @ 800K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=45941674#Test-Agent) (Nature CNN)<br/>
3. [academy_run_to_score_with_keeper @ 800K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=45703399#Test-Agent) (Nature CNN)<br/>
4. [academy_pass_and_shoot_with_keeper @ 800K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=45716494#Test-Agent) (Nature CNN)<br/>
5. [academy_run_pass_and_shoot_with_keeper @ 1.6M steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=46590578#Testing) (Nature CNN)<br/>
6. [academy_3_vs_1_with_keeper @ 500K steps](https://www.kaggle.com/kwabenantim/gfootball-stable-baselines3?scriptVersionId=46843278#Testing) (GFootball CNN)